In [3]:
import numpy as np
import pandas as pd
import scipy.linalg as la

In [9]:
np.random.seed(123)
n = 100
x = np.array([np.random.normal(0, 1, (n,1))]).reshape(-1,1)
theta_0 = np.array([0.0])
p = theta_0.shape[0]
eps = 0.01
eta = 0.001*np.eye(p)
alpha = 0.01*np.eye(p)
V = np.eye(p)
batch_size = n
niter = 500000

$U(\theta) = 2\theta^2+\theta^4$  
$\nabla U(\theta) = -4\theta + 4\theta^3$  
$\nabla\tilde{U}(\theta) = \nabla U(\theta)+\mathcal{N}(0,4) = -4\theta + 4\theta^3 + \mathcal{N}(0,4)$ 

In [10]:
def U(theta):
    return 2*theta**2+theta**4

In [11]:
def gradU_noise(theta, x, n, batch_size):
    '''noisy gradient from paper fig1'''
    return -4*theta + 4*theta**3 + np.random.normal(0,2)

In [12]:
def data_batch(data, batch_size):
    n = data.shape[0]
    p = data.shape[1]
    if n % batch_size !=0:
        n = (n // batch_size)*batch_size
    idx = np.arange(n)
    np.random.shuffle(idx)
    n_batch = n//batch_size
    data = data[idx].reshape(batch_size, p, n_batch)
    return(data, n_batch)

In [13]:
def is_pos_def(A):
    '''function to check if matrix is positive definite'''
    return np.all(np.linalg.eigvals(A) > 0)

In [14]:
def sghmc(gradU, eps, C, Minv, theta_0, V_hat, niter, data, batch_size):
    
    '''Define SGHMC as dscribed in
    Stochastic Gradient Hamilton Monte Carlo, ICML 2014
    Tianqi Chen, Emily B. Fox, Carlos Guestrin.
    
    n: number of observations in data
    p: dimension of parameters
    
    Inputs:
        gradU: function with parameter(theta), gradient of U
        
        eps: learning rate
        
        C: friction matrix, with shape (p,p)
        
        Minv: Mass matrix, with shape (p,p)
        
        theta_0: initial value for sampling
        
        V_hat: estimated covariance matrix of stochastic gradient noise
        
        niter: number of samples to generate
        
        batch_size: size of a minibatch in an iteration
        
    
    Output:
        theta_samp: np.array sampled thetas
    '''
    
    ###Initialization and condition check###
    
    p = len(theta_0)
    n = data.shape[0]
    
    theta_samp = np.zeros((p, niter*(n//batch_size)))
    
    B_hat = 0.5*eps*V_hat
    sqrt_noise = la.sqrtm(2*(C-B_hat)*eps)
    
    theta = theta_0
    sqrtM = la.sqrtm(la.inv(Minv))
    r = np.random.multivariate_normal(np.zeros(p), sqrtM).reshape(p, -1)
    
    j = 0
    for i in range(niter):
        dat_batch, nbatches = data_batch(data, batch_size)
        
        r = np.random.multivariate_normala(np.zeros(p), sqrtM).reshape(p, -1)
        
        for batch in range(nbatches):
            theta = theta + (eps*Minv@r).ravel()
            gradU_batch = gradU(theta, dat_batch[:,:, batch], n, batch_size).reshape(p, -1)
            r = r-eps@gradU_batch - eps*C@Minv@r \
                + np.random.multivariate_normal(np.zeros(p), sqrt_noise).reshape(p, -1)
            
            theta_samp[:,j] = theta
            j = j+1
            
    return theta_samp

In [15]:
def sghmc_momentum(gradU, eta, niter, alpha, theta_0, V_hat, data, batch_size):
    '''Define SGHMC as described in 
    Stochastic Gradient Hamilton Monte Carlo, ICML 2014
    Tianqi Chen, Emily B. Fox, Carlos Guestrin.
    
    n: number of observations in data
    p: dimension of parameters 
    
    Inputs:
        gradU: function with parameter(theta), gradient of U
        
        eta: eps^2 M^(-1) where eps is the learning rate (detail in paper)
        
        niter: number of samples to generate
        
        alpha: eps M^(-1) C where C is a friction term (detail in paper)
        
        V_hat: estimated covariance matrix of stochastic gradient noise
        
        theta_0: initial value for sampling
        
        batch_size: size of a minibatch in an iteration
    
    Out:
        np.array sampled thetas
    '''
    
    ### Initialization and condition check ###
    p = len(theta_0)
    n = data.shape[0]
    
    #set up matrix to store samples
    theta_samp = np.zeros((p, niter*(n//batch_size)))
    
    #beta_hat as p6 mentions
    beta_hat = 0.5*V_hat @ eta
    
    #since we sample from N(0, 2(alpha - beta_hat)eta)
    #the variance matrix must be positive definite
    Sigma = 2*(alpha - beta_hat) @ eta
    
    if not is_pos_def(Sigma):
        print("Error: (alpha - beta_hat) eta is not positive definite")
        return
    
    #batch size <= number of observations
    if batch_size > n:
        print("Error: batch_size must not be larger than number of observations")
        return
    
    #initialize
    theta = theta_0
    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p, -1)
    
    #sampling
    j = 0
    for i in range(niter):
        
        dat_batch, nbatches = data_batch(data, batch_size)
        
        #resample nu every time
        nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p, -1)
        
        for batch in range(nbatches):
            gradU_batch = gradU(theta, dat_batch[:,:, batch], n, batch_size).reshape(p, -1)
            nu = nu-eta@gradU_batch - alpha@nu + np.random.multivariate_normal(np.zeros(p), Sigma).reshape(p, -1)
            theta = theta + nu
            theta_samp[:,j] = theta.T
            j = j+1
            
    return theta_samp

In [16]:
sghmc_momentum(gradU_noise, eta, niter, alpha, theta_0, V, x, batch_size)

array([[-0.06417087, -0.06319911, -0.10133339, ...,  1.09411835,
         1.05997912,  1.05961883]])